In [17]:
import random
import torch
import pandas as pd
import numpy as np

#导入波士顿房价数据集
data = pd.read_csv('../boston_housing_data.csv')
selected_features = data[['RM', 'DIS', 'LSTAT', 'TAX']]
selected_features = data.select_dtypes(include=[np.number]).columns.tolist() #只要数字
selected_labels = data['MEDV']
selected_labels = data.select_dtypes(include=[np.number]).columns.tolist()
data = data.dropna()
# 提取选择的特征和标签
# features = torch.tensor(data.drop('MEDV', axis=1).values, dtype=torch.float32)
# labels = torch.tensor(data['MEDV'].values, dtype=torch.float32).reshape((-1, 1))
features = torch.tensor(data[selected_features].values, dtype=torch.float32)
labels = torch.tensor(data[selected_labels].values, dtype=torch.float32)

# 修改模型的输入和参数：
w = torch.normal(0, 0.01, size=(len(selected_features), 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# print(features)
# print(labels)
print(data)
print(selected_labels)

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
447  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273   
448  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273   
449  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273   
450  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273   
451  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273   

     PIRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396.90   9.14  21.6  
2       17.8  3

In [21]:
#读取数据集：模型训练的时候需要对数据集进行遍历，每一次都取一个小批量样本，并且使用它们更新模型
#随机读取一个小批量样本

def data_iter(batch_size, features, labels):
    num_examples=len(features)
    indices=list(range(num_examples))
    #打乱列表顺序 实现随机读取
    random.shuffle(indices)
    #按batch_size读取数据集
    for i in range(0, num_examples, batch_size):
        batch_indices=torch.tensor(indices[i:min(i+batch_size,num_examples)]) #避免batch_size越界
        yield features[batch_indices],labels[batch_indices] #返回读取的数据 保存当前的位置 下次继续迭代
    
# for batch_features, batch_labels in data_iter(10, features, labels):
#     print("features:",batch_features, "\nlabels:",batch_labels)

def linreg(X,w,b):
    """线性回归模型"""
#     return torch.matmul(X,w)+b
#     print(w)
    return torch.mm(X,w)+b

# print(linreg(X,w,b))

#定义损失函数
def squared_loss(y_hat,y):
    """均方损失函数"""
    #y.reshape(y_hat.shape) 将y改造成y_hat一样的矩阵 才能加减
#     return (y_hat-y.reshape(y_hat.shape))**2 / 2
    return torch.mean((y_hat - y)**2) / 2

#定义优化算法：线性回归有解析解
#设置指定的训练次数，在每次训练中，从数据集随机取一个小批量，然后根据参数计算损失函数的梯度。
#然后，朝着减少损失的方向更新参数
def sgd(params,lr,batch_size):
    """小批量随机梯度下降""" 
    #设置学习率：lr(步长)， batch_size：批次大小
    #with :==try catch:不管是否出现异常 都自动回收资源
    #因为计算的损失是一个批量样本的总和，所以可以考虑使用批量大小为步长，刚好可以抵消
    #with torch.no_grad(): 是一个上下文管理器，用于控制是否计算并存储梯度信息。在这个上下文中，所有的操作都不会生成梯度(自动求导)，也不会进行梯度计算。
    with torch.no_grad(): 
        for param in params:
            #通过将参数的梯度乘以学习率，并除以批次大小，得到更新的大小，然后从参数的当前值中减去该更新大小。grad：求梯度(求导)
            param -= lr * param.grad / batch_size
            param.grad.zero_() #将参数的梯度清零，准备进行下一轮的梯度计算和更新。

# 设置学习率和训练批次大小(超参数)
lr = 0.06
batch_size = 32
loss = squared_loss
net=linreg

# 训练模型
# 训练次数
num_epochs = 100
for epoch in range(num_epochs):
    #批量读取
    for X,Y in data_iter(batch_size,features,labels):
         # 前向传播：计算模型的预测值
        print( w,b)
        y_hat = net(X, w, b)
        l=loss(y_hat,Y) #计算XY的小批量损失值
        
        # 反向传播：计算损失函数关于模型参数的梯度
        l.mean().backward()
        # RuntimeError: grad can be implicitly created only for scalar outputs
        # 出现了非标量（non-scalar）损失函数的情况。PyTorch的自动微分机制要求梯度计算是基于标量的，因此无法为非标量张量创建梯度。选择 'mean' 或 'sum' 可以将损失降维为标量。
        
        #计算关于[w,b]的梯度,更新模型参数
        sgd([w,b],lr,batch_size)
        
        # 重置参数的梯度
        w.grad.zero_()
        b.grad.zero_()
        
#     #展示训练精度
#     with torch.no_grad():
#         train_l=loss(net(features,w,b),labels)
#         print(f"epoch {epoch+1},loss {float(train_l.mean()):f}")
#         #随着训练次数增加 w和b越来越接近真实值
#         print("w:",end='')
#         for wi in w:
#             print(wi.item(),end=' ')
#         print(f"b:{b.item()}")
#         #item将张量转化成标量输出
            
# print(f"真实值：w:{true_w},b:{true_b}")

tensor([[-4.5931e-03],
        [ 2.4088e-03],
        [ 2.3939e-02],
        [-3.1189e-03],
        [ 6.7697e-03],
        [ 1.3889e-02],
        [-5.5421e-05],
        [ 7.9528e-04],
        [ 1.9770e-02],
        [ 5.1742e-03],
        [ 9.9892e-04],
        [-1.6292e-02],
        [-6.7634e-03],
        [ 1.0112e-02]], requires_grad=True) tensor([0.], requires_grad=True)
tensor([[1.6179e-01],
        [2.0256e+00],
        [1.2742e+00],
        [5.1247e-04],
        [7.1989e-02],
        [8.1730e-01],
        [7.7956e+00],
        [4.8765e-01],
        [9.5800e-01],
        [4.5316e+01],
        [2.2471e+00],
        [4.6921e+01],
        [1.3464e+00],
        [3.1027e+00]], requires_grad=True) tensor([0.1250], requires_grad=True)
tensor([[-8.8218e+01],
        [-6.8739e+02],
        [-6.3546e+02],
        [-2.8987e+00],
        [-3.4764e+01],
        [-4.1282e+02],
        [-4.2031e+03],
        [-2.8364e+02],
        [-5.6142e+02],
        [-2.5347e+04],
        [-1.1917e+03],
     

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n

        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],


tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True) tensor([nan], requires_grad=True)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [n